In [1]:
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

from sklearn.model_selection import train_test_split
from pylab import *

import pickle
import loadnotebook
from predictionhelper import *

from bayes_opt import BayesianOptimization
import sklearn.metrics as metric

import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from predictionhelper.ipynb


In [55]:
sets = [x for x in range(1, 34)]
demo_config = {6 : sets}

df_all_data = get_data(config=demo_config, pure=True, refresh=False).reset_index(drop=True)
print(len(df_all_data))

1113700


In [56]:
prediction_columns = ["RSRP", "RSRQ", "SNR"]
pred_index = 2
pred_col = prediction_columns[pred_index]
group = ['location_x', 'location_y', 'priority', 'set', 'PCI']
group2 = ['location_x', 'location_y', 'priority', 'set']

In [57]:
dropped_columns = [c for c in df_all_data if "beam" in c] + prediction_columns[pred_index+1:]
df_data = df_all_data.drop(dropped_columns, axis=1)
df_data = df_data[df_data["PCI"].isin(whitelist_PCI)]
df_data = df_data.dropna()

In [52]:
# df_data = df_data.drop(['PCI']+prediction_columns[:pred_index], axis=1)
# df_data = df_data.drop_duplicates().reset_index(drop=True)

In [58]:
df_data.groupby(group).count()
# df_data.groupby(group2).count()

RSRP  RSRQ  SNR  Power_37  Power_38  \
location_x location_y priority set PCI                                        
110.0      217.0      6        1   37    303   303  303       303       303   
111.0      201.0      6        2   37    286   286  286       286       286   
           203.0      6        14  37     21    21   21        21        21   
           206.0      6        11  37    109   109  109       109       109   
112.0      184.0      6        32  37      7     7    7         7         7   
           199.0      6        18  37    170   170  170       170       170   
           201.0      6        29  37    111   111  111       111       111   
           203.0      6        32  37     53    53   53        53        53   
           220.0      6        32  37    134   134  134       134       134   
           221.0      6        18  37    126   126  126       126       126   
113.0      184.0      6        6   37      3     3    3         3         3   
                               12  37     26    26   26        26        26   
                               28  37     13    13   13        13        13   
           200.0      6        17  37     62    62   62        62        62   
           201.0      6        8   37    199   199  199       199       199   
           202.0      6        12  37    294   294  294       294       294   
                               28  37     64    64   64        64        64   
           203.0      6        10  37     53    53   53        53        53   
           219.0      6        17  37    165   165  165       165       165   
           220.0      6        10  37     46    46   46        46        46   
                               11  37     33    33   33        33        33   
                               20  37     85    85   85        85        85   
                               28  37    168   168  168       168       168   
           221.0      6        12  37    594   594  594       594       594   
114.0      185.0      6        13  37     11    11   11        11        11   
           201.0      6        20  37     53    53   53        53        53   
           202.0      6        13  37    239   239  239       239       239   
           203.0      6        6   37     81    81   81        81        81   
           220.0      6        13  37    111   111  111       111       111   
           221.0      6        2   37    216   216  216       216       216   
...                                      ...   ...  ...       ...       ...   
915.0      161.0      6        27  42    206   206  206       206       206   
                               28  42    114   114  114       114       114   
           162.0      6        2   42    394   394  394       394       394   
                               3   42    382   382  382       382       382   
                               8   42    349   349  349       349       349   
                               11  42    141   141  141       141       141   
                               18  42    156   156  156       156       156   
                               21  42    155   155  155       155       155   
                               31  42    119   119  119       119       119   
                               32  42    122   122  122       122       122   
                               33  42    206   206  206       206       206   
           163.0      6        17  42    137   137  137       137       137   
           164.0      6        22  42    137   137  137       137       137   
           175.0      6        9   42    134   134  134       134       134   
                               10  42    144   144  144       144       144   
                               18  42    155   155  155       155       155   
                               23  42    132   132  132       132       132   
                               24  42    104   104  104       104       104   
           

In [59]:
df_data = merge_agg(df_data, group, pred_col, ['count'])
new_cols = [x+"_mean" for x in prediction_columns[:pred_index+1]]
df_data = merge_agg(df_data, group, prediction_columns[:pred_index+1], ['mean'], new_cols)
idx = df_data.groupby(group2)['count'].transform(max) == df_data['count']
df_data = df_data[idx].reset_index(drop=True)

# df_data = merge_agg(df_data, group2, prediction_columns[pred_index], ['mean'], ["SNR_mean"])

In [60]:
df_data.groupby(group).count()
# df_data.groupby(group2).count()

RSRP  RSRQ  SNR  Power_37  Power_38  \
location_x location_y priority set PCI                                        
110.0      217.0      6        1   37    303   303  303       303       303   
111.0      201.0      6        2   37    286   286  286       286       286   
           203.0      6        14  37     21    21   21        21        21   
           206.0      6        11  37    109   109  109       109       109   
112.0      184.0      6        32  37      7     7    7         7         7   
           199.0      6        18  37    170   170  170       170       170   
           201.0      6        29  37    111   111  111       111       111   
           203.0      6        32  37     53    53   53        53        53   
           220.0      6        32  37    134   134  134       134       134   
           221.0      6        18  37    126   126  126       126       126   
113.0      184.0      6        6   37      3     3    3         3         3   
                               12  37     26    26   26        26        26   
                               28  37     13    13   13        13        13   
           200.0      6        17  37     62    62   62        62        62   
           201.0      6        8   37    199   199  199       199       199   
           202.0      6        12  37    294   294  294       294       294   
                               28  37     64    64   64        64        64   
           203.0      6        10  37     53    53   53        53        53   
           219.0      6        17  37    165   165  165       165       165   
           220.0      6        10  37     46    46   46        46        46   
                               11  37     33    33   33        33        33   
                               20  37     85    85   85        85        85   
                               28  37    168   168  168       168       168   
           221.0      6        12  37    594   594  594       594       594   
114.0      185.0      6        13  37     11    11   11        11        11   
           201.0      6        20  37     53    53   53        53        53   
           202.0      6        13  37    239   239  239       239       239   
           203.0      6        6   37     81    81   81        81        81   
           220.0      6        13  37    111   111  111       111       111   
           221.0      6        2   37    216   216  216       216       216   
...                                      ...   ...  ...       ...       ...   
915.0      161.0      6        27  42    206   206  206       206       206   
                               28  42    114   114  114       114       114   
           162.0      6        2   42    394   394  394       394       394   
                               3   42    382   382  382       382       382   
                               8   42    349   349  349       349       349   
                               11  42    141   141  141       141       141   
                               18  42    156   156  156       156       156   
                               21  42    155   155  155       155       155   
                               31  42    119   119  119       119       119   
                               32  42    122   122  122       122       122   
                               33  42    206   206  206       206       206   
           163.0      6        17  42    137   137  137       137       137   
           164.0      6        22  42    137   137  137       137       137   
           175.0      6        9   42    134   134  134       134       134   
                               10  42    144   144  144       144       144   
                               18  42    155   155  155       155       155   
                               23  42    132   132  132       132       132   
                               24  42    104   104  104       104       104   
           

In [61]:
for i in range(pred_index+1) :
    df_data[prediction_columns[i]] = df_data[prediction_columns[i] + "_mean"]
    
df_data = df_data.drop(new_cols+['count'], axis=1)
df_data = df_data.drop_duplicates()

# df_data[pred_col] = df_data[pred_col + "_mean"]    
# df_data = df_data.drop(['SNR_mean'], axis=1)
# df_data = df_data.drop_duplicates()

In [62]:
df_data.columns, len(df_data)

(Index(['location_x', 'location_y', 'PCI', 'RSRP', 'RSRQ', 'SNR', 'Power_37',
        'Power_38', 'Power_39', 'Power_40', 'Power_41', 'Power_42',
        'Distance_37', 'Distance_38', 'Distance_39', 'Distance_40',
        'Distance_41', 'Distance_42', 'Angle_37', 'Angle_38', 'Angle_39',
        'Angle_40', 'Angle_41', 'Angle_42', 'priority', 'set'],
       dtype='object'), 6701)

In [63]:
data_train, data_test = pd.DataFrame(), pd.DataFrame()
data_train_dict, data_test_dict = {}, {}
for p in demo_config :
    for s in demo_config[p] :
        a, b = train_test_split(df_data[df_data.set==s], test_size=0.3, random_state=32)
        data_train = data_train.append(a)
        data_test = data_test.append(b)   
        data_train_dict[(p, s)] = a
        data_test_dict[(p, s)] = b
print(len(data_train), len(data_test))

exclude_train_col = ['priority', pred_col]
x_train = data_train.drop(exclude_train_col, axis=1)
y_train = np.array(data_train[pred_col].values.tolist())
x_test = data_test.drop(exclude_train_col, axis=1)
y_test = np.array(data_test[pred_col].values.tolist())

x_train_dict, y_train_dict, x_test_dict, y_test_dict = {}, {}, {}, {}
for p in demo_config :
    for s in demo_config[p] :
        a, b = data_train_dict[(p,s)], data_test_dict[(p,s)]
        x_train_dict[(p, s)] = a.drop(exclude_train_col, axis=1)
        y_train_dict[(p, s)] = np.array(a[pred_col].values.tolist())
        x_test_dict[(p, s)] = b.drop(exclude_train_col, axis=1)
        y_test_dict[(p, s)] = np.array(b[pred_col].values.tolist())

4674 2027


In [66]:
print('rsrp', max(data_test['RSRP']), min(data_test['RSRP']))
print('rsrq', max(data_test['RSRQ']), min(data_test['RSRQ']))
print('snr', max(data_test['SNR']), min(data_test['SNR']))

rsrp -57.84772727272729 -120.2375
rsrq -5.5 -20.984375
snr 29.9017167381974 -8.8


In [67]:
print('rsrp', 0.07*(max(data_test['RSRP']) - min(data_test['RSRP'])))
print('rsrq', 0.1*(max(data_test['RSRQ']) - min(data_test['RSRQ'])))
print('snr', 0.09*(max(data_test['SNR']) - min(data_test['SNR'])))

rsrp 4.36728409090909
rsrq 1.5484375000000001
snr 3.4831545064377663


# SVM

In [12]:
from sklearn import svm

svm_params = {'kernel':'rbf'}

class svm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {'kernel':'poly', 'degree':3, 'gamma':'auto', 'coef0':0.0, 'tol':0.001}
        params = {'kernel':'rbf', 'degree':3, 'gamma':'auto', 'coef0':0.0, 'tol':0.001}
        params['C'] = param['C'] if param['C'] > 0 else 0.1
        params['epsilon'] = param['epsilon'] if param['epsilon'] > 0 else 0.001
        return params
        
    def evaluate(self, degree=3, gamma='auto_deprecated', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1):
        params = {'degree':degree, 'gamma':gamma, 'coef0':coef0, 'tol':tol, 'C':C, 'epsilon':epsilon}
        params = self.clean_param(params)

        model =svm.SVR(**params)
        model.fit(self.x_train, self.y_train)
        y_pred = model.predict(self.x_test)
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(self.y_test, predictions)
        rmse = math.sqrt(mse)
        return -1*rmse

In [11]:
# st = svm_target(x_train, y_train, 
#                 x_test, y_test)
# sBO = BayesianOptimization(st.evaluate, {'C': (0.001, 1), 'epsilon' : (0, 0.1)},
#                             random_state = 1)

# sBO.maximize(init_points=20, n_iter=5)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         C |   epsilon | 
    1 | 00m01s |  -12.31807 |    0.4170 |    0.0000 | 
    2 | 00m01s |  -12.31807 |    0.7203 |    0.0000 | 
    3 | 00m01s |  -12.31818 |    0.0001 |    0.0000 | 
    4 | 00m01s |  -12.31807 |    0.3023 |    0.0000 | 
    5 | 00m01s |  -12.31818 |    0.1468 |    0.0000 | 
    6 | 00m01s |  -12.31818 |    0.0923 |    0.0000 | 
    7 | 00m01s |  -12.31818 |    0.1863 |    0.0000 | 
    8 | 00m01s |  -12.31807 |    0.3456 |    0.0000 | 
    9 | 00m01s |  -12.31807 |    0.3968 |    0.0000 | 
   10 | 00m01s |  -12.31807 |    0.5388 |    0.0000 | 
   11 | 00m01s |  -12.31807 |    0.4192 |    0.0000 | 
   12 | 00m01s |  -12.31807 |    0.6852 |    0.0000 | 
   13 | 00m01s |  -12.31807 |    0.2045 |    0.0000 | 
   14 | 00m01s |  -12.31694 |    0.8781 |    0.0000 | 
   15 | 00m01s |  -12.31818 |    0.0274 |    0.0000 | 
   16 | 00m01s |  -12.31807 |    0.6705 |    0.0000

In [ ]:
# svm_params = {'kernel':'rbf'}
# model = svm.SVR(**svm_params)
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# predictions = [round(value) for value in y_pred]
# mse = metric.mean_squared_error(y_test, predictions)
# rmse = math.sqrt(mse)/(max(y_test)-min(y_test))
# rmse

# KNN 

In [13]:
from sklearn.neighbors import KNeighborsRegressor

knn_params_list = [
{'n_neighbors': len(whitelist_PCI),
 'weights': 'distance',
 'algorithm': 'brute',
 'leaf_size': 5,
 'p': 1}, 
{'n_neighbors': len(whitelist_PCI),
 'weights': 'uniform',
 'algorithm': 'kd_tree',
 'leaf_size': 49,
 'p': 1},
{'n_neighbors': len(whitelist_PCI),
 'weights': 'uniform',
 'algorithm': 'kd_tree',
 'leaf_size': 9,
 'p': 1}
]

knn_params = knn_params_list[pred_index]

class knn_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.weights = ['uniform', 'distance']
        self.algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']
        
    def clean_param(self, param) :
        params = {'n_neighbors':7}
        params['weights'] = self.weights[int(param['weight'])]
        params['algorithm'] = self.algorithms[int(param['algorithm'])]
        params['leaf_size'] = int(param['leaf_size'])
        params['p'] = int(param['p'])
        return params
        
    def evaluate(self, weight, algorithm, leaf_size=100, p=2):
        params = {}
        params['weight'] = weight
        params['algorithm'] = algorithm
        params['leaf_size'] = int(leaf_size)
        params['p'] = int(p)
        params = self.clean_param(params)

        model = KNeighborsRegressor(**params)
        model.fit(self.x_train, self.y_train)
        y_pred = model.predict(self.x_test)
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(self.y_test, predictions)
        rmse = math.sqrt(mse)
        return -1*rmse

In [17]:
# kt = knn_target(x_train, y_train, 
#                 x_test, y_test)
# kBO = BayesianOptimization(kt.evaluate, {'weight': (0, 1),
#                                         'algorithm' : (0, 3),
#                                         'leaf_size' : (5, 50),
#                                         'p': (1, 2),},
#                             random_state = 1)

# kBO.maximize(init_points=20, n_iter=5)

Initialization
---------------------------------------------------------------------------------
 Step |   Time |      Value |   algorithm |   leaf_size |         p |    weight | 
    1 | 00m00s |   -7.57702 |      2.4022 |     49.4987 |    1.1023 |    0.4170 | 
    2 | 00m00s |   -7.57702 |      2.9048 |     38.6675 |    1.4141 |    0.7203 | 
    3 | 00m00s |   -7.57008 |      0.9403 |     17.6200 |    1.6944 |    0.0001 | 
    4 | 00m00s |   -7.57702 |      2.0770 |     40.5176 |    1.4142 |    0.3023 | 
    5 | 00m00s |   -7.57008 |      2.6292 |      9.6452 |    1.0500 |    0.1468 | 
    6 | 00m00s |   -7.57163 |      2.6838 |     25.1552 |    1.5359 |    0.0923 | 
    7 | 00m00s |   -7.57702 |      0.2551 |     45.8868 |    1.6638 |    0.1863 | 
    8 | 00m00s |   -7.57163 |      0.1172 |     18.2126 |    1.5149 |    0.3456 | 
    9 | 00m00s |   -7.57008 |      0.5095 |     17.9499 |    1.9446 |    0.3968 | 
   10 | 00m00s |   -7.57008 |      2.6344 |     10.8513 |    1.5866 |    

In [14]:
# params = kt.clean_param(kBO.res['max']['max_params'])
# params

In [16]:
# model = KNeighborsRegressor(**knn_params)
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# predictions = [round(value) for value in y_pred]
# mse = metric.mean_squared_error(y_test, predictions)
# rmse = math.sqrt(mse)/(max(y_test)-min(y_test))
# rmse

0.10514341524899071

# XGBoost  

In [14]:
from xgboost import XGBRegressor

xgboost_params_list = [
# RSRP,
{'base_score': 0.5,
'booster': 'gbtree',
'missing': None,
'n_estimators': 100,
'n_jobs': 1,
'random_state': 1,
'reg_lambda': 1,
'alpha': 0,
'scale_pos_weight': 1,
'subsample': 1,
'colsample_bytree': 1,
'colsample_bylevel': 1,
'objective': 'reg:linear',
'learning_rate': 0.06926984074036927,
'gamma': 43.90712517147065,
'max_depth': 9,
'min_child_weight': 1,
'max_delta_weight': 14,
'rate_drop': 0.5865550405019929},
# RSRQ
{'base_score': 0.5,
 'booster': 'dart',
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'random_state': 1,
 'reg_lambda': 1,
 'alpha': 0,
 'scale_pos_weight': 1,
 'subsample': 1,
 'colsample_bytree': 1,
 'colsample_bylevel': 1,
 'objective': 'reg:linear',
 'learning_rate': 0.12,
 'gamma': 0.0,
 'max_depth': 3,
 'min_child_weight': 1,
 'max_delta_weight': 20,
 'rate_drop': 0.0},
# SNR
{'base_score': 0.5,
 'booster': 'dart',
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'random_state': 1,
 'reg_lambda': 1,
 'alpha': 0,
 'scale_pos_weight': 1,
 'subsample': 1,
 'colsample_bytree': 1,
 'colsample_bylevel': 1,
 'objective': 'reg:linear',
 'learning_rate': 0.12,
 'gamma': 0.0,
 'max_depth': 12,
 'min_child_weight': 1,
 'max_delta_weight': 20,
 'rate_drop': 0.0}
]

xgboost_params = xgboost_params_list[pred_index]

class xgboost_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        booster_dict = {1:'gbtree', 2:'gblinear', 3:'dart'}
        params = {'base_score':0.5, 'booster':'gbtree', 'missing':None, 'n_estimators':100, 
                  'n_jobs':1, 'random_state':1, 'reg_lambda':1, 'alpha':0, 'scale_pos_weight':1, 
                  'subsample':1, 'colsample_bytree':1, 'colsample_bylevel':1}
        
        params['objective'] = 'reg:linear'
        params['learning_rate'] = param['learning_rate']/100
        params['booster'] = booster_dict[int(param['booster'])]
        params['gamma'] = param['gamma']
        params['max_depth'] = int(param['max_depth'])
        params['min_child_weight'] = int(param['min_child_weight'])
        params['max_delta_weight'] = int(param['max_delta_weight'])
        params['rate_drop'] = param['rate_drop']
        return params
        
    def evaluate(self, learning_rate, booster, gamma, max_depth,  
                     min_child_weight, max_delta_weight, rate_drop):

        params = {}
        params['learning_rate'] = learning_rate
        params['booster'] = booster
        params['gamma'] = gamma
        params['max_depth'] = max_depth
        params['min_child_weight'] = min_child_weight
        params['max_delta_weight'] = max_delta_weight
        params['rate_drop'] = rate_drop
        params = self.clean_param(params)

        xgb_model = XGBRegressor(**params)
        xgb_model.fit(self.x_train, self.y_train)
        y_pred = xgb_model.predict(self.x_test)
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(self.y_test, predictions)
        rmse = math.sqrt(mse)
        return -1*rmse

In [15]:
# xt = xgboost_target(x_train, y_train, x_test, y_test)
# xgbBO = BayesianOptimization(xt.evaluate, {'learning_rate': (1, 12),
#                                             'booster' : (1, 3),
#                                             'gamma' : (0, 50),
#                                             'max_depth': (3, 12),
#                                             'min_child_weight': (1, 1),
#                                             'max_delta_weight': (1, 20),
#                                             'rate_drop': (0, 1)},
#                             random_state = 1)

# xgbBO.maximize(init_points=10, n_iter=5)

In [98]:
# params = xt.clean_param(xgbBO.res['max']['max_params'])
# xgb_model = XGBRegressor(**params)
# xgb_model.fit(x_train, y_train)

# y_pred = xgb_model.predict(x_test)
# predictions = [round(value) for value in y_pred]
# mse = metric.mean_squared_error(y_test, predictions)
# rmse = math.sqrt(mse)
# print(rmse)

In [17]:
# params

In [20]:
# xgb_model = XGBRegressor(**xgboost_params)
# xgb_model.fit(x_train, y_train)

# y_pred = xgb_model.predict(x_test)
# predictions = [round(value) for value in y_pred]
# mse = metric.mean_squared_error(y_test, predictions)
# rmse = math.sqrt(mse)/(max(y_test)-min(y_test))
# print(rmse)

0.05673710121377912


# LGBM 

In [15]:
import lightgbm
import lightgbm as lgb
from lightgbm import LGBMRegressor

lgbm_params_list = [
# RSRP
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'min_child_samples': 20,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'objective': None,
 'random_state': 0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'num_leaves': 50,
 'min_child_weight': 0,
 'max_depth': -1,
 'learning_rate': 0.1,
 'min_data_in_bin': 1,
 'min_data': 1},
# RSRQ
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'min_child_samples': 20,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'objective': None,
 'random_state': 0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'num_leaves': 19,
 'min_child_weight': 0,
 'max_depth': 5,
 'learning_rate': 0.0995815310228581,
 'min_data_in_bin': 1,
 'min_data': 1},
# SNR
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'min_child_samples': 20,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'objective': None,
 'random_state': 0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'num_leaves': 32,
 'min_child_weight': 0,
 'max_depth': 11,
 'learning_rate': 0.1,
 'min_data_in_bin': 1,
 'min_data': 1}
]

lgbm_params = lgbm_params_list[pred_index]

class lgbm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {'boosting_type':'gbdt', 'class_weight':None, 'colsample_bytree':1.0, 
                  'importance_type':'split', 
                  'min_child_samples':20, 'min_split_gain':0.0, 'n_estimators':100, 'objective':None,
                  'random_state':0, 'reg_alpha':0.0, 'reg_lambda':0.0, 'silent':True,
                  'subsample':1.0, 'subsample_for_bin':200000, 'subsample_freq':0}
        params['num_leaves'] = int(param['num_leaves'])
        params['min_child_weight'] = int(param['min_child_weight'])
        params['max_depth'] = int(param['max_depth'])
        params['learning_rate'] = param['learning_rate'] / 100
        params['min_data_in_bin'] = 1
        params['min_data'] = 1
        return params
        
    def evaluate(self, min_child_weight, learning_rate, max_depth, num_leaves):
        params = {'num_leaves':num_leaves, 
                  'min_child_weight':min_child_weight, 
                  'max_depth':max_depth, 
                  'learning_rate':learning_rate}
        
        params = self.clean_param(params)

        lgbm_model = LGBMRegressor(**params)
        lgbm_model.fit(self.x_train, self.y_train)
        y_pred = lgbm_model.predict(self.x_test)
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(self.y_test, predictions)
        rmse = math.sqrt(mse)
        return -1*rmse

In [22]:
# lt = lgbm_target(x_train, y_train, x_test, y_test)
# lgbmBO = BayesianOptimization(lt.evaluate, {'min_child_weight': (0.01, 1),
#                                               'learning_rate': (1, 10),
#                                               'max_depth': (-1, 15),
#                                               'num_leaves': (5, 50)}, 
#                              random_state=3)

# lgbmBO.maximize(init_points=20, n_iter=5)

In [23]:
# params = lt.clean_param(lgbmBO.res['max']['max_params'])
# lgbm_model = LGBMRegressor(**params)
# lgbm_model.fit(x_train, y_train)
# y_pred = lgbm_model.predict(x_test)
# predictions = [round(value) for value in y_pred]
# mse = metric.mean_squared_error(y_test, predictions)
# rmse = math.sqrt(mse)
# print(rmse)

In [24]:
# params

In [24]:
# lgbm_model = LGBMRegressor(**lgbm_params)
# lgbm_model.fit(x_train, y_train)
# y_pred = lgbm_model.predict(x_test)
# predictions = [round(value) for value in y_pred]
# mse = metric.mean_squared_error(y_test, predictions)
# rmse = math.sqrt(mse)/(max(y_test)-min(y_test))
# print(rmse)

0.05499986297950971


# Experiment 

In [16]:
nrmse_matrix = np.empty((4, 3, 34))
nrmse_matrix[:] = np.nan

In [17]:
model_name_list = ['xgboost', 'lgbm', 'knn', 'svm']

def get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test) :
    if 'xgb' in model_name : 
        return xgboost_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    elif 'knn' in model_name : 
        return knn_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    elif 'lgbm' in model_name : 
        return lgbm_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    else :
        return svm_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    
def get_params_range(model_name) :
    if 'xgb' in model_name : 
        return {'learning_rate': (1, 12),
                'booster' : (1, 3),
                'gamma' : (0, 5),
                'max_depth': (3, 10),
                'min_child_weight': (1, 1),
                'max_delta_weight': (1, 12),
                'rate_drop': (0, 1)}
    elif 'knn' in model_name :
        return {'weight': (0, 1),
                'algorithm' : (0, 3),
                'leaf_size' : (5, 50),
                'p': (1, 2),}
    elif 'lgbm' in model_name :
        return {'min_child_weight': (0.01, 1),
              'learning_rate': (1, 10),
              'max_depth': (-1, 15),
              'num_leaves': (5, 50)} 
    else :
        return {'C': (0.01, 1), 'epsilon' : (0.001, 0.1)}
    
def reset_model(model_name, params=None) :
    if 'xgb' in model_name :
        return XGBRegressor(**xgboost_params) if params is None else XGBRegressor(**params)
    elif 'knn' in model_name :
        return KNeighborsRegressor(**knn_params) if params is None else KNeighborsRegressor(**params)
    elif 'svm' in model_name :
        return svm.SVR(**svm_params) if params is None else svm.SVR(**params)
    else :
        return LGBMRegressor(**lgbm_params) if params is None else LGBMRegressor(**params)

# Manual Experiment 

In [45]:
model_idx = 0
model_name = model_name_list[model_idx]

# Baseline 

In [46]:
pred_index, model_name

(2, 'xgboost')

In [64]:
random = 0
t = get_target(model_name, x_train, y_train, x_test, y_test)
xgbBO = BayesianOptimization(t.evaluate, 
                             get_params_range(model_name),
                             random_state = random, 
                             verbose=0)

xgbBO.maximize(init_points=20, n_iter=3)
params = t.clean_param(xgbBO.max['params'])
model = reset_model(model_name, params=params)

# model = reset_model(model_name)
model.fit(x_train, y_train)

for s in demo_config[6] :
    y_pred = model.predict(x_test_dict[(6, s)])
    predictions = [round(value) for value in y_pred]
    mse = metric.mean_squared_error(y_test_dict[(6, s)], predictions)
    nrmse = math.sqrt(mse)/(max(y_test_dict[(6, s)]) - min(y_test_dict[(6, s)]))
    print(nrmse)
#     nrmse_matrix[model_idx, 0, s] = nrmse
        
# pickle.dump(model, open("db/%s_%s_baseline.pickle.dat" % (pred_col, model_name), "wb"))
# for x in nrmse_matrix[model_idx, 0, 1:]:
#     print('%.3f' % x)

0.09717551602107012
0.06268658898100694
0.07598806600786864
0.08523501676600076
0.07736309533120861
0.11573036652202501
0.10636166024605741
0.07067199944075084
0.07625369534924394
0.08963427796907403
0.055980853574525986
0.10762109400090289
0.10313847606763601
0.12591736815811894
0.1303506180085006
0.07436204917097626
0.09354021894380607
0.08961868881761961
0.08578317686645091
0.08949570437992846
0.12416410605801646
0.06358109512739067
0.11156460691683817
0.10334584400715428
0.0958040315515723
0.12103456077859076
0.10970337637575837
0.09414509670386624
0.051546233547065994
0.06553755190013637
0.08436657998044483
0.06698688662778085
0.1097387093990368


# Independent 

In [130]:
for s in demo_config[6] :
    t = get_target(model_name, x_train_dict[(6, s)], y_train_dict[(6, s)], 
                   x_test_dict[(6, s)], y_test_dict[(6, s)])
    xgbBO = BayesianOptimization(t.evaluate, 
                                 get_params_range(model_name),
                                 random_state = random, 
                                 verbose=0)

    xgbBO.maximize(init_points=20, n_iter=3)
    params = t.clean_param(xgbBO.res['max']['max_params'])
    model = reset_model(model_name, params=params)

#     model = reset_model(model_name)
    model.fit(x_train_dict[(6, s)], y_train_dict[(6, s)])

    y_pred = model.predict(x_test_dict[(6, s)])
    predictions = [round(value) for value in y_pred]
    mse = metric.mean_squared_error(y_test_dict[(6, s)], predictions)
    nrmse = math.sqrt(mse)/(max(y_test_dict[(6, s)]) - min(y_test_dict[(6, s)]))
    nrmse_matrix[model_idx, 1, s] = nrmse
        
    pickle.dump(model, open("db/%s_%s_independent_set_%s.pickle.dat" % (pred_col, model_name, s), "wb"))
    
for x in nrmse_matrix[model_idx, 1, 1:]:
    print('%.3f' % x)

0.223
0.209
0.201
0.211
0.240
0.225
0.234
0.261
0.229
0.223
0.232
0.240
0.194
0.236
0.219
0.189
0.290
0.226
0.203
0.232
0.205
0.215
0.215
0.201
0.214
0.214
0.240
0.210
0.173
0.215
0.208
0.215
0.233


# Transfer

In [131]:
for s in demo_config[6] :
    curr_x_train = df_data[df_data.set!=s].drop(exclude_train_col, axis=1)
    curr_y_train = df_data[df_data.set!=s][pred_col].values.tolist()
    curr_x_test = df_data[df_data.set==s].drop(exclude_train_col, axis=1)
    curr_y_test = df_data[df_data.set==s][pred_col].values.tolist()

    t = get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    xgbBO = BayesianOptimization(t.evaluate, 
                                 get_params_range(model_name),
                                 random_state = random, 
                                 verbose=0)

    xgbBO.maximize(init_points=20, n_iter=3)
    params = t.clean_param(xgbBO.res['max']['max_params'])
    model = reset_model(model_name, params=params)
    
#     model = reset_model(model_name)
    model.fit(curr_x_train, curr_y_train)

    y_pred = model.predict(curr_x_test)
    predictions = [round(value) for value in y_pred]
    mse = metric.mean_squared_error(curr_y_test, predictions)
    nrmse = math.sqrt(mse)/(max(curr_y_test) - min(curr_y_test))
    nrmse_matrix[model_idx, 2, s] = nrmse

    pickle.dump(model, open("db/%s_%s_transfer_except_%s.pickle.dat" % (pred_col, model_name, s), "wb"))
    
for x in nrmse_matrix[model_idx, 2, 1:]:
    print('%.3f' % x)

0.217
0.224
0.179
0.190
0.209
0.248
0.207
0.231
0.224
0.204
0.208
0.221
0.185
0.200
0.214
0.209
0.215
0.204
0.215
0.234
0.193
0.192
0.180
0.186
0.178
0.217
0.193
0.187
0.174
0.178
0.175
0.193
0.179


# Automatic Experiment 

In [65]:
random = 0
for model_idx in range(0, 2) :
    model_name = model_name_list[model_idx]
    print(model_name)
    
#     Baseline
    t = get_target(model_name, x_train, y_train, x_test, y_test)
    xgbBO = BayesianOptimization(t.evaluate, 
                                 get_params_range(model_name),
                                 random_state = random, 
                                 verbose=0)

    xgbBO.maximize(init_points=20, n_iter=3)
    params = t.clean_param(xgbBO.max['params'])
    model = reset_model(model_name, params=params)
    model.fit(x_train, y_train)

    print('============')
    for s in demo_config[6] :
        y_pred = model.predict(x_test_dict[(6, s)])
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(y_test_dict[(6, s)], predictions)
        nrmse = math.sqrt(mse)/(max(y_test_dict[(6, s)]) - min(y_test_dict[(6, s)]))
        nrmse_matrix[model_idx, 0, s] = nrmse
        print(nrmse)

#     pickle.dump(model, open("db/%s_%s_baseline.pickle.dat" % (pred_col, model_name), "wb"))
    
# Independent
    print('============')
    for s in demo_config[6] :
        t = get_target(model_name, x_train_dict[(6, s)], y_train_dict[(6, s)], 
                       x_test_dict[(6, s)], y_test_dict[(6, s)])
        xgbBO = BayesianOptimization(t.evaluate, 
                                     get_params_range(model_name),
                                     random_state = random, 
                                     verbose=0)

        xgbBO.maximize(init_points=20, n_iter=3)
        params = t.clean_param(xgbBO.max['params'])
        model = reset_model(model_name, params=params)
        model.fit(x_train_dict[(6, s)], y_train_dict[(6, s)])

        y_pred = model.predict(x_test_dict[(6, s)])
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(y_test_dict[(6, s)], predictions)
        nrmse = math.sqrt(mse)/(max(y_test_dict[(6, s)]) - min(y_test_dict[(6, s)]))
        nrmse_matrix[model_idx, 1, s] = nrmse
        print(nrmse)

#         pickle.dump(model, open("db/%s_%s_independent_set_%s.pickle.dat" % (pred_col, model_name, s), "wb"))

# Transfer
    for s in demo_config[6] :
        curr_x_train = df_data[df_data.set!=s].drop(exclude_train_col, axis=1)
        curr_y_train = df_data[df_data.set!=s][pred_col].values.tolist()
        curr_x_test = df_data[df_data.set==s].drop(exclude_train_col, axis=1)
        curr_y_test = df_data[df_data.set==s][pred_col].values.tolist()

        t = get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test)
        xgbBO = BayesianOptimization(t.evaluate, 
                                     get_params_range(model_name),
                                     random_state = random, 
                                     verbose=0)

        xgbBO.maximize(init_points=20, n_iter=3)
        params = t.clean_param(xgbBO.max['params'])
        model = reset_model(model_name, params=params)
        model.fit(curr_x_train, curr_y_train)

        y_pred = model.predict(curr_x_test)
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(curr_y_test, predictions)
        nrmse = math.sqrt(mse)/(max(curr_y_test) - min(curr_y_test))
        nrmse_matrix[model_idx, 2, s] = nrmse

#         pickle.dump(model, open("db/%s_%s_transfer_except_%s.pickle.dat" % (pred_col, model_name, s), "wb"))

xgboost
0.09717551602107012
0.06268658898100694
0.07598806600786864
0.08523501676600076
0.07736309533120861
0.11573036652202501
0.10636166024605741
0.07067199944075084
0.07625369534924394
0.08963427796907403
0.055980853574525986
0.10762109400090289
0.10313847606763601
0.12591736815811894
0.1303506180085006
0.07436204917097626
0.09354021894380607
0.08961868881761961
0.08578317686645091
0.08949570437992846
0.12416410605801646
0.06358109512739067
0.11156460691683817
0.10334584400715428
0.0958040315515723
0.12103456077859076
0.10970337637575837
0.09414509670386624
0.051546233547065994
0.06553755190013637
0.08436657998044483
0.06698688662778085
0.1097387093990368


KeyboardInterrupt: 

# Model Analysis 

In [ ]:
scenarios = ['baseline', 'independent', 'transfer']
for scenario in range(3) :
    print("========================================"+scenarios[scenario])
    baseline_metrics = nrmse_matrix[:, scenario, 1:]
    baseline_metrics = baseline_metrics[~np.isnan(baseline_metrics)].reshape((4,len(sets)))
    print('avg of each methods', [ '%.3f' % elem for elem in np.mean(baseline_metrics, axis=1)])
    print('best performance', np.unique(np.argmin(baseline_metrics, axis=0), return_counts=True))
    
    print('diff performance')
    min_list = np.min(baseline_metrics, axis=0)

    for x in baseline_metrics :
        diff = x-min_list
        print('%.3f' % np.mean(diff[np.nonzero(diff)]))

# Scenario Analysis

In [ ]:
for curr_model in range(4) :
    print("========================================"+model_name_list[curr_model])
    metrics = nrmse_matrix[curr_model, :, 1:]
    metrics = metrics[~np.isnan(metrics)].reshape((3,len(sets)))
    print('avg of each scenario', [ '%.3f' % elem for elem in np.mean(metrics, axis=1)])
    print('best performance', np.unique(np.argmin(metrics, axis=0), return_counts=True))
    
    transfer_metrics = nrmse_matrix[curr_model, 2, 1:]
    for i in range(2) :
        curr_metrics = nrmse_matrix[curr_model, i, 1:]
        print('[%s]diff avg %.3f' % (scenarios[i], 
                                        np.mean(curr_metrics - transfer_metrics)))
        
        improve_idx = transfer_metrics < curr_metrics
        diff = transfer_metrics[improve_idx] - curr_metrics[improve_idx]
        print('[%s]+++++++increase %d %.3f' % (scenarios[i], len(diff), np.mean(diff)))
        
        drop_idx = transfer_metrics > curr_metrics
        diff = transfer_metrics[drop_idx] - curr_metrics[drop_idx]
        print('[%s]-------decrease %d %.3f' % (scenarios[i], len(diff), np.mean(diff)))

In [177]:
nrmse_matrix = nrmse_matrix_snr

In [58]:
# nrmse_matrix_snr = nrmse_matrix 
# save_to_pickle(nrmse_matrix_snr, 'nrmse_matrix_snr')

In [125]:
# nrmse_matrix_rsrq = nrmse_matrix 
# save_to_pickle(nrmse_matrix_rsrq, 'nrmse_matrix_rsrq')

In [35]:
# nrmse_matrix_rsrp = nrmse_matrix 
# save_to_pickle(nrmse_matrix_rsrp, 'nrmse_matrix_rsrp')

In [179]:
test = load_from_pickle('nrmse_matrix_snr')

In [180]:
for x in test[3, 2, 1:]:
    print('%.3f' % x)

0.218
0.135
0.204
0.203
0.224
0.234
0.208
0.205
0.263
0.284
0.222
0.244
0.178
0.220
0.262
0.276
0.324
0.176
0.198
0.281
0.294
0.274
0.292
0.234
0.274
0.248
0.321
0.216
0.182
0.210
0.230
0.177
0.259


In [178]:
for x in nrmse_matrix[3, 2, 1:]:
    print('%.3f' % x)

0.218
0.135
0.204
0.203
0.224
0.234
0.208
0.205
0.263
0.284
0.222
0.244
0.178
0.220
0.262
0.276
0.324
0.176
0.198
0.281
0.294
0.274
0.292
0.234
0.274
0.248
0.321
0.216
0.182
0.210
0.230
0.177
0.259


# Generate Predicted Coordinate 

In [166]:
x_cut = 50  
y_cut = 100 

old_origin_img = cv2.imread('../image/map.png',0)
crop = old_origin_img[y_cut:318, x_cut:927]
crop = cv2.cvtColor(crop, cv2.COLOR_GRAY2RGB)

x_coord_list = []
y_coord_list = []
pci_list = []
for lon in range(0, crop.shape[1]) :
    for lat in range(0, crop.shape[0]) :
        x_coord_list.append(x_cut + lon)
        y_coord_list.append(y_cut + lat)
        
all_x_pci = pd.DataFrame({'location_x':x_coord_list, 'location_y':y_coord_list})

# Bayesian Opt - Exploration - Partial Point

In [167]:
from matplotlib import gridspec

class target() :
    def optimize(self, x, y) :
        if self.bayes_opt is None or self.bayes_opt.X is None:
            return -1000

        bo = self.bayes_opt
        bo.gp.fit(bo.X, bo.Y)
        mu, sigma = bo.gp.predict(all_x_pci.values, return_std=True)
        return -mean(sigma)

def posterior(bo, x):
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma = bo.gp.predict(x, return_std=True)
    plot(sigma)
    plt.show()
    return mu, sigma

def plot_gp(bo, x, curr_x_train, curr_y_train, set_val, model, show_sigma_map=False):
#     background = get_map_image()
#     pci_val = [get_pci(d[0], d[1]) for d in bo.X]
#     observation_color = [pci_color_dict[y] if y in pci_color_dict else (255, 255, 255) for y in pci_val]
#     path = "../results/predicted/pci/bayesian_%s_set_%d.png" % (model, set_val)
#     a = visualize(background, bo.X[:, 0].astype(int), bo.X[:, 1].astype(int), 
#                   observation_color, path, adjustment=True)
    
    path = "../results/predicted/pci/real_%s_set_%d.png" % (model, set_val)
    background = get_map_image()
    p_color = [pci_decode[y] for y in curr_y_train]
    p_color = [pci_color_dict[y] if y in pci_color_dict else (255, 255, 255) for y in p_color]
    b = visualize(background, curr_x_train['location_x'].astype(int), curr_x_train['location_y'].astype(int), 
                  p_color, path, adjustment=True)

    if show_sigma_map :
        normalize_sigma = matplotlib.colors.Normalize(vmin=min(sigma), vmax=max(sigma))
        mu_map = [cmap(normalize_sigma(value))[:3] for value in mu_sigma]
        mu_map = [[int(x*255) for x in value] for value in mu_map]    
        a=visualize_all_location_heatmap(a, x_coord_view, y_coord_view, mu_map, 
                                         cmap, normalize_sigma, filename=None,
                                         size=1, figsize=(20,10), adjustment=False, show=False)

In [168]:
class target2() :
    def __init__(self, set_val) :
        self.set_val = set_val
    
    def optimize(self, x, y) :
        if self.bayes_opt is None or self.bayes_opt.X is None or len(self.bayes_opt.X) < 2:
            return -1000

        curr_df_data = df_data[df_data.set == self.set_val]

        temp = curr_df_data.copy()
        temp2 = pd.DataFrame(columns=curr_df_data.columns)
    
        bo = self.bayes_opt
        for x in bo.X :
            distance = lambda d: math.hypot(abs(x[0]-d[0]), abs(x[1]-d[1]))
            temp["d"] = temp.apply(distance, axis=1)
            temp2 = temp2.append(temp.loc[temp.d.idxmin()])

        temp3 = curr_df_data[~curr_df_data.index.isin(temp2.index)]
        temp2 = curr_df_data[~curr_df_data.index.isin(temp3.index)]

        curr_x_train = temp2.drop(exclude_train_col, axis=1)
        curr_y_train = np.array(temp2[pred_col].values.tolist())
        curr_x_test = temp3.drop(exclude_train_col, axis=1)
        curr_y_test = np.array(temp3[pred_col].values.tolist())

        params = {'min_child_weight': 0.7151893663724195, 'learning_rate': 4.9382849013642325, 
                  'max_depth': 7.462318716046472, 'num_leaves': 5.909827884814657,
                  'min_data':1, 'min_data_in_bin':1}
        t = get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test)
        params = t.clean_param(params)

        model = reset_model(model_name, params)
        model.fit(curr_x_train, curr_y_train)

        y_pred = model.predict(curr_x_test)
        predictions = [round(value) for value in y_pred]
        mse = metric.mean_squared_error(curr_y_test, predictions)
        rmse = math.sqrt(mse)
        return -1*rmse

In [36]:
# acc_dict = {}
# # for set_val in demo_config[6] :
# random = 0
# t = target2(2)
# bo2 = BayesianOptimization(t.optimize, {'x': (min(x_coord_list), max(x_coord_list)), 
#                                         'y': (min(y_coord_list), max(y_coord_list))},
#                            random_state=random, 
#                            verbose=1)
# t.bayes_opt = bo2

# iterations = 50
# gp_params = {"alpha": 1e-5, "n_restarts_optimizer": 3, 'random_state':random}
# bo2.maximize(init_points=10, n_iter=iterations, acq="ei", xi=1e+2, **gp_params)
# #     bo2.maximize(init_points=2, n_iter=iterations, acq="ei", xi=1e-4, **gp_params)
# acc_dict[set_val] = bo2.res['max']['max_val']
# print(acc_dict[set_val])

In [37]:
# bayes_spec_target_inden = np.array([x for x in acc_dict.values()])
# for x in list(bayes_spec_target_inden[:, 2]) :
#     print(x)

# Target : Variance 

In [169]:
random = 0
t = target()
bo2 = BayesianOptimization(t.optimize, {'x': (min(x_coord_list), max(x_coord_list)), 
                                        'y': (min(y_coord_list), max(y_coord_list))},
                           random_state=random, 
                           verbose=1)
t.bayes_opt = bo2

iterations = 500
gp_params = {"alpha": 1e-5, "n_restarts_optimizer": 3, 'random_state':random}
bo2.maximize(init_points=2, n_iter=iterations, acq="ei", xi=1e+2, **gp_params)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
    1 | 00m00s | 1000.00000 |  530.7606 |  230.7997 | 
    2 | 00m00s |   -0.99999 |  676.5059 |  218.2397 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
    3 | 00m00s |   -0.99998 |  689.4791 |  189.5332 | 
    4 | 00m00s |   -0.98814 |  518.5220 |  146.1513 | 
    5 | 00m00s |   -0.99514 |   72.7881 |  228.1264 | 
    6 | 00m00s |   -0.99392 |  617.4562 |  261.6594 | 
    7 | 00m00s |   -0.99309 |  837.4735 |  285.0548 | 
    8 | 00m00s |   -0.99193 |   61.4990 |  274.8175 | 
    9 | 00m00s |   -0.99060 |  702.4929 |  159.9914 | 
   10 | 00m00s |   -0.98828 |  463.5776 |  267.2542 | 
   11 | 00m00s |   -0.99007 |  645.7080 |  121.2919 | 
   12 | 00m00s |   -0.98907 |  270.6671 |  108.0514 | 
   13 | 00m00s |   -0.98819 |  110.9284 |  167.7725 | 
   14 | 00m00s |   -0.98718 | 

KeyboardInterrupt: 

# Bayesian Independent 

In [80]:
acc_dict = {}
for set_val in demo_config[6] :
    curr_df_data = df_data[df_data.set == set_val]
    iterations = int(0.5*len(curr_df_data))

    temp = curr_df_data.copy()
    temp2 = pd.DataFrame(columns=curr_df_data.columns)
    
#     random = 0
#     t = target()
#     bo2 = BayesianOptimization(t.optimize, {'x': (min(x_coord_list), max(x_coord_list)), 
#                                             'y': (min(y_coord_list), max(y_coord_list))},
#                                random_state=random, 
#                                verbose=1)
#     t.bayes_opt = bo2
#     gp_params = {"alpha": 1e-5, "n_restarts_optimizer": 3, 'random_state':random}
#     bo2.maximize(init_points=2, n_iter=iterations, acq="ei", xi=0.1, **gp_params)
#     bo2.maximize(init_points=2, n_iter=iterations, acq="poi", xi=0.1, **gp_params)
#     bo2.maximize(init_points=2, n_iter=iterations, acq="ucb", kappa=10, **gp_params)
    
    for x in bo2.X[:iterations] :
        distance = lambda d: math.hypot(abs(x[0]-d[0]), abs(x[1]-d[1]))
        temp["d"] = temp.apply(distance, axis=1)
        temp2 = temp2.append(temp.loc[temp.d.idxmin()])

#     trouble_col = ['PCI', 'Power_37', 'Power_38', 'Power_39', 'Power_40', 'Power_41', 'Power_42', 'set']
#     temp2 = temp2.astype({x:'int' for x in trouble_col})
    temp3 = curr_df_data[~curr_df_data.index.isin(temp2.index)]
    temp2 = curr_df_data[~curr_df_data.index.isin(temp3.index)]

#     curr_x_train = temp2.drop(["d"] + exclude_train_col, axis=1)
    curr_x_train = temp2.drop(exclude_train_col, axis=1)

    curr_y_train = np.array(temp2[pred_col].values.tolist())
    curr_x_test = temp3.drop(exclude_train_col, axis=1)
    curr_y_test = np.array(temp3[pred_col].values.tolist())

    t = get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    bo = BayesianOptimization(t.evaluate, 
                              get_params_range(model_name),
                              random_state = random, 
                              verbose=0)

    bo.maximize(init_points=5, n_iter=1)
    params = t.clean_param(bo.res['max']['max_params'])
#     params = lgbm_params
    
    model = reset_model(model_name, params)
    model.fit(curr_x_train, curr_y_train)

    y_pred = model.predict(curr_x_test)
    predictions = [round(value) for value in y_pred]
    mse = metric.mean_squared_error(curr_y_test, predictions)
    rmse = math.sqrt(mse)
    print(rmse, bo.res['max']['max_params'])
    acc_dict[set_val] = [len(curr_x_train), len(curr_x_test), rmse]
    pickle.dump(model, open("db/%s_%s_bayesian_independent_%s.pickle.dat" % \
                            (pred_col, model_name, set_val), "wb"))

4.821738512324429 {'min_child_weight': 0.5488135039273248, 'learning_rate': 6.813047017599905, 'max_depth': 11.667600609322633, 'num_leaves': 8.920818486569331}
5.869447942718806 {'min_child_weight': 0.5488135039273248, 'learning_rate': 6.813047017599905, 'max_depth': 11.667600609322633, 'num_leaves': 8.920818486569331}
6.810767097323696 {'min_child_weight': 0.0, 'learning_rate': 10.0, 'max_depth': 15.0, 'num_leaves': 5.0}
6.676304747455873 {'min_child_weight': 0.7151893663724195, 'learning_rate': 4.9382849013642325, 'max_depth': 7.462318716046472, 'num_leaves': 5.909827884814657}
10.942097447853618 {'min_child_weight': 0.0, 'learning_rate': 10.0, 'max_depth': 15.0, 'num_leaves': 5.0}
6.876517988770098 {'min_child_weight': 0.5488135039273248, 'learning_rate': 6.813047017599905, 'max_depth': 11.667600609322633, 'num_leaves': 8.920818486569331}
5.890160185507073 {'min_child_weight': 0.5488135039273248, 'learning_rate': 6.813047017599905, 'max_depth': 11.667600609322633, 'num_leaves': 8.9

In [41]:
bayes_inden = np.array([x for x in acc_dict.values()])
for x in list(bayes_inden[:, 2]) :
    print(x)

4.821845259171405
5.798746431639859
6.425932530646177
6.392791663987237
10.338726783903612
6.993589816543581
5.6533314347618076
7.1338953661280495
6.5164719173273165
5.5422780149075335
7.423855646125879
7.210860002019517
7.908292051971078
7.02496775169628
6.254003229848743
6.063140947819642
7.448805244743188
6.0175956670192345
5.2863935526125525
7.452567203195131
7.173078174239747
7.299732528452897
5.447505811954517
6.529144750576596
6.578250714082722
5.408482976147868
6.122643140128807
6.041555969223726


# Bayesian Baseline 

In [42]:
acc_dict = {}
all_curr_x_train, all_curr_y_train = pd.DataFrame(), []
all_curr_x_test_dict, all_curr_y_test_dict = {}, {}
for set_val in demo_config[6] :
    curr_df_data = df_data[df_data.set == set_val]
    iterations = int(0.2*len(curr_df_data))

    temp = curr_df_data.copy()
    temp2 = pd.DataFrame(columns=curr_df_data.columns)
    for x in bo2.X[:iterations] :
        distance = lambda d: math.hypot(abs(x[0]-d[0]), abs(x[1]-d[1]))
        temp["d"] = temp.apply(distance, axis=1)
        temp2 = temp2.append(temp.loc[temp.d.idxmin()])

#     trouble_col = ['PCI', 'Power_37', 'Power_38', 'Power_39', 'Power_40', 'Power_41', 'Power_42', 'set']
#     temp2 = temp2.astype({x:'int' for x in trouble_col})
    temp3 = curr_df_data[~curr_df_data.index.isin(temp2.index)]
    temp2 = curr_df_data[~curr_df_data.index.isin(temp3.index)]

#     curr_x_train = temp2.drop(["d"] + exclude_train_col, axis=1)
    curr_x_train = temp2.drop(exclude_train_col, axis=1)
    curr_y_train = temp2[pred_col].values.tolist()
    curr_x_test = temp3.drop(exclude_train_col, axis=1)
    curr_y_test = temp3[pred_col].values.tolist()
    
    all_curr_x_train = all_curr_x_train.append(curr_x_train)
    all_curr_y_train += curr_y_train 
    all_curr_x_test_dict[set_val] = curr_x_test
    all_curr_y_test_dict[set_val] = curr_y_test  

#     plot_gp(bo2, all_x_pci.values, curr_x_train, curr_y_train, set_val, "xgboost")
    
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':4.2522, 
#               'max_delta_weight':11, 'random_state' :random}
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':1, 
#               'max_delta_weight':11, 'random_state' :random}

t = get_target(model_name, all_curr_x_train, all_curr_y_train, all_curr_x_test, all_curr_y_test)
xgbBO = BayesianOptimization(t.evaluate, 
                             get_params_range(model_name),
                             random_state = random, 
                             verbose=0)

xgbBO.maximize(init_points=5, n_iter=3)
print(xgbBO.res['max']['max_params'])
params = t.clean_param(xgbBO.res['max']['max_params'])

# params = lgbm_params
# params['min_data_in_bin']=1
# params['min_data']=1
    
model = reset_model(model_name, params)
model.fit(curr_x_train, curr_y_train)
pickle.dump(model, open("db/%s_%s_bayesian_%s.pickle.dat" % (pred_col, model_name, set_val), "wb"))

for set_val in demo_config[6] :
    y_pred = model.predict(all_curr_x_test_dict[set_val])
    predictions = [round(value) for value in y_pred]
    mse = metric.mean_squared_error(all_curr_y_test_dict[set_val], predictions)
    rmse = math.sqrt(mse)
#     print(rmse)    
    acc_dict[set_val] = [len(curr_x_train), len(curr_x_test), rmse]
    pickle.dump(model, open("db/%s_%s_bayesian_baseline_set_%s.pickle.dat" % (pred_col, model_name, s), "wb"))

NameError: name 'all_curr_x_test' is not defined

In [ ]:
bayes_baseline = np.array([x for x in acc_dict.values()])
for x in list(bayes_baseline[:, 2]) :
    print(x)

# Bayesian Transfer 

In [ ]:
acc_dict = {}
all_curr_x_train_dict, all_curr_y_train_dict = {}, {}
all_curr_x_test_dict, all_curr_y_test_dict = {}, {}
for set_val in demo_config[6] :
    curr_df_data = df_data[df_data.set == set_val]
    iterations = int(0.2*len(curr_df_data))

    temp = curr_df_data.copy()
    temp2 = pd.DataFrame(columns=curr_df_data.columns)
    for x in bo2.X[:iterations] :
        distance = lambda d: math.hypot(abs(x[0]-d[0]), abs(x[1]-d[1]))
        temp["d"] = temp.apply(distance, axis=1)
        temp2 = temp2.append(temp.loc[temp.d.idxmin()])

#     trouble_col = ['PCI', 'Power_37', 'Power_38', 'Power_39', 'Power_40', 'Power_41', 'Power_42', 'set']
#     temp2 = temp2.astype({x:'int' for x in trouble_col})
    temp3 = curr_df_data[~curr_df_data.index.isin(temp2.index)]
    temp2 = curr_df_data[~curr_df_data.index.isin(temp3.index)]

#     curr_x_train = temp2.drop(["d"] + exclude_train_col, axis=1)
    curr_x_train = temp2.drop(exclude_train_col, axis=1)
    curr_y_train = temp2[pred_col].values.tolist()
    
    all_curr_x_train_dict[set_val] = curr_x_train
    all_curr_y_train_dict[set_val] = curr_y_train  
    all_curr_x_test_dict[set_val] = curr_df_data.drop(exclude_train_col, axis=1)
    all_curr_y_test_dict[set_val] = curr_df_data[pred_col].values.tolist()  

#     plot_gp(bo2, all_x_pci.values, curr_x_train, curr_y_train, set_val, "xgboost")
    
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':4.2522, 
#               'max_delta_weight':11, 'random_state' :random}
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':1, 
#               'max_delta_weight':11, 'random_state' :random}

params = lgbm_params
params['min_data_in_bin']=1
params['min_data']=1
    
for set_val in demo_config[6] :
    curr_x_train, curr_y_train = pd.DataFrame(), []
    for k in all_curr_x_train_dict :
        if k != set_val :
            curr_x_train = curr_x_train.append(all_curr_x_train_dict[k])
            curr_y_train += all_curr_y_train_dict[k]

    t = get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    xgbBO = BayesianOptimization(t.evaluate, 
                                 get_params_range(model_name),
                                 random_state = random, 
                                 verbose=0)

    xgbBO.maximize(init_points=5, n_iter=3)
    print(xgbBO.res['max']['max_params'])
    params = t.clean_param(xgbBO.res['max']['max_params'])

    model = reset_model(model_name, params)
    model.fit(curr_x_train, curr_y_train)
    pickle.dump(model, open("db/%s_%s_bayesian_transfer_%s.pickle.dat" % ('PCI', model_name, set_val), "wb"))
    
    y_pred = model.predict(all_curr_x_test_dict[set_val])
    predictions = [round(value) for value in y_pred]
    mse = metric.mean_squared_error(all_curr_y_test_dict[set_val], predictions)
    rmse = math.sqrt(mse)
    print(rmse)
    acc_dict[set_val] = [len(curr_x_train), len(curr_x_test), rmse]

In [ ]:
bayes_transfer = np.array([x for x in acc_dict.values()])
for x in list(bayes_transfer[:, 2]) :
    print(x)